<a href="https://colab.research.google.com/github/jacr2006/android-tflitemodel-drowsiness-detection/blob/master/data_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorboardcolab

In [3]:
import numpy as np
from tensorboardcolab import TensorBoardColab

Using TensorFlow backend.


In [29]:
N_LANDMARK = 68# numero de marcas faciales
INPUT_SHAPE = (96, 96, 1)# ancho, alto y canales de la imagen
OUTPUT_SIZE = N_LANDMARK*2# tamaño de la capa de salida, dos coordenadas por cada marca facial

# lectura de los dataser previamente procesados
PATH_DRIVE = '/content/drive/My Drive/Proyectos Nuevos/ML/multiclass_landmark_keras_cnn/'
X = np.load(PATH_DRIVE + "img_dataset.npy")
y = np.load(PATH_DRIVE + "pts_dataset.npy")
print(y.shape)
y = y.reshape(-1, OUTPUT_SIZE)

# escalada de la data de entrada
#m = np.mean(X)
#X = (X-m)/255.0# normalizacion de los datos entre (-1,1)
X = X/255.0#normalizacion de los datos entre (0,1)

print(X.shape)
print(y.shape)


(6666, 68, 2)
(6666, 96, 96, 1)
(6666, 136)


Visualizacion de la data normalizada

In [30]:
print(X[100,:,:,0])

[[0.15686275 0.1254902  0.10588235 ... 0.07843137 0.1372549  0.18039216]
 [0.16470588 0.11372549 0.11372549 ... 0.08235294 0.12156863 0.18039216]
 [0.14509804 0.10588235 0.12941176 ... 0.07843137 0.10588235 0.16470588]
 ...
 [0.78431373 0.17647059 0.21176471 ... 0.16862745 0.12156863 0.23137255]
 [0.80392157 0.20784314 0.23529412 ... 0.81176471 0.65098039 0.34901961]
 [0.83921569 0.25490196 0.23137255 ... 0.65490196 0.60784314 0.69411765]]


In [31]:
print(y[100,:])

[  4.  27.   5.  38.   8.  51.   9.  63.  12.  74.  18.  82.  28.  91.
  39.  99.  50. 100.  60.  98.  67.  91.  73.  84.  78.  74.  82.  64.
  84.  53.  85.  41.  86.  29.  15.  19.  19.  10.  28.  10.  38.  13.
  45.  17.  58.  16.  66.  14.  72.  11.  79.   9.  83.  16.  52.  25.
  53.  34.  54.  43.  54.  52.  43.  59.  48.  60.  52.  61.  56.  60.
  60.  58.  23.  26.  29.  23.  35.  23.  39.  29.  34.  30.  28.  30.
  60.  29.  65.  24.  72.  23.  75.  27.  72.  30.  65.  30.  34.  69.
  42.  69.  46.  69.  49.  70.  53.  69.  58.  70.  65.  69.  57.  76.
  52.  79.  48.  79.  45.  78.  41.  76.  37.  69.  45.  72.  49.  73.
  53.  73.  62.  71.  53.  73.  49.  73.  45.  72.]


Estructura de la red neuronal (CNN)

In [0]:
import keras
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, AveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation, Dense
from keras.layers import Flatten, Input, Convolution2D, add, Dropout, MaxPool2D
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=INPUT_SHAPE))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(OUTPUT_SIZE))

In [38]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.optimizers import Adam
import numpy as np
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

NUM_EPOCHS = 100# numero de epocas de entrenamiento
INIT_LR = 0.001# coeficiente de aprendizaje inicial
BS = 256# se inicio el entrenamiento con un tamaño de 32 y se aumento hasta lograr la convergencia

# entrenar una CNN para detectar marcas faciales es distinto de predecir clases (binaria o multiclase)
# en vez de obtener una distribucion de clases usando una funcion de perdida como "entropy loss" que mide la capacidad de clasificacion,
# se debe usar una fucnion de perdida para regresion, la cual compara directamente el valor inferido con el real 

#opt = SGD(lr=INIT_LR, momentum=0.9)# Gradiente descendete estoscastico como metodo de optimizacion
opt = Adam(lr=INIT_LR, decay=1e-3 / 200)

# compilacion del modelo
model.compile(loss='mean_squared_error', optimizer=opt, metrics=["mae"])

tbc=TensorBoardColab()
callbacks_list = [TensorBoardColabCallback(tbc)]


Wait for 8 seconds...
TensorBoard link:
https://b783080e.ngrok.io


In [39]:
H = model.fit(
    X, y, batch_size=BS, 
    epochs=NUM_EPOCHS, shuffle=True,
    verbose=1, 
    validation_split = 0.2,
    callbacks=callbacks_list)

model.save(PATH_DRIVE+'models/model2.h5')


Train on 5332 samples, validate on 1334 samples
Epoch 1/100
5332/5332 [==============================] - 6s 1ms/step - loss: 28270.4508 - mean_absolute_error: 16.9843 - val_loss: 719.0173 - val_mean_absolute_error: 11.5242
Epoch 2/100
5332/5332 [==============================] - 4s 741us/step - loss: 22589.1363 - mean_absolute_error: 16.9909 - val_loss: 685.9101 - val_mean_absolute_error: 10.7773
Epoch 3/100
5332/5332 [==============================] - 4s 742us/step - loss: 16263.7122 - mean_absolute_error: 14.5716 - val_loss: 1234.7526 - val_mean_absolute_error: 11.3038
Epoch 4/100
5332/5332 [==============================] - 4s 739us/step - loss: 8981.5007 - mean_absolute_error: 13.3318 - val_loss: 887.2483 - val_mean_absolute_error: 10.8456
Epoch 5/100
5332/5332 [==============================] - 4s 741us/step - loss: 9856.4670 - mean_absolute_error: 13.1393 - val_loss: 5269.7010 - val_mean_absolute_error: 22.3118
Epoch 6/100
5332/5332 [==============================] - 4s 743us/ste